In [ ]:
# When using a colab notebook:
#!wget https://github.com/Aleph-Alpha/examples/blob/main/bootcamp/data.md
#!wget https://github.com/Aleph-Alpha/examples/blob/main/requirements.txt
#!pip install -r requirements.txt

In [1]:
from aleph_alpha_client import Client, Prompt, CompletionRequest,ExplanationRequest, TextControl, EvaluationRequest, TargetGranularity
from scipy import spatial
import numpy as np
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

client = Client(token=os.getenv("AA_TOKEN"))

#### Lets learn about Attention Manipulation

In [8]:
text = "The quick brown fox jumps over the lazy dog.\nThe color of the fox is"
# Here we define a TextControl that will be used to control the attention on the prompt.
# Change the factor to 0.0 to see what happens.
control = TextControl(start=10, length=5, factor=1)
prompt = Prompt.from_text(text, controls=[control])

request = CompletionRequest(prompt=prompt, maximum_tokens=10, stop_sequences=["."])
result = client.complete(request = request, model="luminous-extended")
print(result.completions[0].completion)

 brown


We see, that changing the attention changes the output of the model. Let's see how that can help us in different scenarios.

In [13]:
# Let's try that again. but this time, we want to get traceable explanations.
text = "The quick brown fox jumps over the lazy dog.\nThe color of the fox is"

controls = []
# create a control for each word in the prompt
for i, word in enumerate(text.split()):
    # we want to control the attention on each word
    # so we set the factor to 1.0
    #get the starting index of the word
    start = text.find(word)
    control = TextControl(start=start, length=len(word), factor=0.1)
    controls.append(control)
    
eval_scores = []
for control in controls:
    prompt = Prompt.from_text(text, controls=[control])
    
    request = EvaluationRequest(prompt=prompt, completion_expected=" brown")
    score = client.evaluate(request = request, model="luminous-extended")
    print(f"The control of '{text[control.start:control.start+control.length]}' is: {score.result['log_perplexity']}")
    


The control of 'The' is: 0.40005174
The control of 'quick' is: 0.80100304
The control of 'brown' is: 2.042087
The control of 'fox' is: 0.318715
The control of 'jumps' is: 0.83131087
The control of 'over' is: 0.67064124
The control of 'the' is: 0.8609614
The control of 'lazy' is: 0.5153156
The control of 'dog.' is: 1.3308904
The control of 'The' is: 0.40005174
The control of 'color' is: 1.5616225
The control of 'of' is: 1.9225018
The control of 'the' is: 0.8609614
The control of 'fox' is: 0.318715
The control of 'is' is: 0.79551744


### Let's use an explantation request to find out what the model is looking at

In [14]:
exp_req = ExplanationRequest(Prompt.from_text(text), " brown", control_factor=0.3)
response_explain = client.explain(exp_req, model="luminous-extended")

explanations = response_explain[1][0].items[0][0]

for item in explanations:
    start = item.start
    end = item.start + item.length
    print(f"""EXPLAINED TEXT: {text[start:end]}
Score: {np.round(item.score, decimals=3)}""")

EXPLAINED TEXT: The
Score: -0.292
EXPLAINED TEXT: quick
Score: 0.117
EXPLAINED TEXT: brown
Score: 1.318
EXPLAINED TEXT: fox
Score: -0.435
EXPLAINED TEXT: jumps
Score: 0.143
EXPLAINED TEXT: over
Score: -0.01
EXPLAINED TEXT: the
Score: 0.16
EXPLAINED TEXT: lazy
Score: -0.255
EXPLAINED TEXT: dog
Score: -0.253
EXPLAINED TEXT: .
Score: 0.627
EXPLAINED TEXT: 

Score: 0.033
EXPLAINED TEXT: The
Score: 0.006
EXPLAINED TEXT: color
Score: 0.773
EXPLAINED TEXT: of
Score: 1.253
EXPLAINED TEXT: the
Score: 0.789
EXPLAINED TEXT: fox
Score: 2.387
EXPLAINED TEXT: is
Score: 0.097


In [15]:
# Read the data in the data.md file
with open("data.md", "r") as f:
    data = f.read()
    
# Split the data into a list of texts
texts = data.split("#####")

print(f"data: {data[:100]}")
print(f"texts: {texts[10][:100]}")

data: SUMMARY

# Most OECD countries have developed and implemented national sustainable development strat
texts:  INDICATORS AND TARGETS

The development and incorporation of quantitative indicators can help remov


In [16]:
answers_prompt = f"""### Instructions: Solve the task based on the text below".

### Input:
{texts[4]}

### Task: Give me a list of countries that include social elements.

### Reponse:"""

response = client.complete(CompletionRequest(prompt=Prompt.from_text(answers_prompt), maximum_tokens=100, stop_sequences=["###"]), model="luminous-base-control")
answer = response.completions[0].completion

exp_req = ExplanationRequest(Prompt.from_text(answers_prompt), answer, control_factor=0.1, prompt_granularity="paragraph", target_granularity=TargetGranularity.Complete)
response_explain = client.explain(exp_req, model="luminous-extended-control")

explanations = response_explain[1][0].items[0][0]

for item in explanations:
    start = item.start
    end = item.start + item.length
    print(f"""EXPLAINED TEXT: {answers_prompt[start:end]}
Score: {np.round(item.score, decimals=3)}""")

EXPLAINED TEXT: ### Instructions: Solve the task based on the text below".
Score: -0.854
EXPLAINED TEXT: ### Input:
 POLICY INTEGRATION
Score: -0.107
EXPLAINED TEXT: POLICY INTEGRATION – **15**
Score: 0.255
EXPLAINED TEXT: Sustainable development has been defined a variety of ways, but in practice it has come to mean development that achieves a balance among economic, environmental and social objectives for both present and future generations. The integration of the three dimensions of sustainable development is one of the most difficult balances to achieve in formulating a national strategy. In practice, most national strategies have a greater focus on environmental issues with some attempts to incorporate economic aspects. The social pillar has been the most neglected. As a result, few national strategies develop abilities for considering and making trade-offs among the three areas in overall policy-making.
Score: -0.091
EXPLAINED TEXT: Most countries emphasize environmental componen

In [17]:
exp_req = ExplanationRequest(Prompt.from_text(answers_prompt), " The USA have social elements.", control_factor=0.1, prompt_granularity="paragraph", target_granularity=TargetGranularity.Complete)
response_explain = client.explain(exp_req, model="luminous-extended")

explanations = response_explain[1][0].items[0][0]

for item in explanations:
    start = item.start
    end = item.start + item.length
    print(f"""EXPLAINED TEXT: {answers_prompt[start:end]}
Score: {np.round(item.score, decimals=3)}""")

EXPLAINED TEXT: ### Instructions: Solve the task based on the text below".
Score: -1.146
EXPLAINED TEXT: ### Input:
 POLICY INTEGRATION
Score: -0.185
EXPLAINED TEXT: POLICY INTEGRATION – **15**
Score: 0.16
EXPLAINED TEXT: Sustainable development has been defined a variety of ways, but in practice it has come to mean development that achieves a balance among economic, environmental and social objectives for both present and future generations. The integration of the three dimensions of sustainable development is one of the most difficult balances to achieve in formulating a national strategy. In practice, most national strategies have a greater focus on environmental issues with some attempts to incorporate economic aspects. The social pillar has been the most neglected. As a result, few national strategies develop abilities for considering and making trade-offs among the three areas in overall policy-making.
Score: -0.224
EXPLAINED TEXT: Most countries emphasize environmental component